In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch import nn

from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import DataLoader

import time

In [2]:
%run '0_tools.ipynb'

In [3]:
use_updated_word_dict = True

In [4]:
if (use_updated_word_dict):
    word_dict = list(np.load(f'{train_vect_path}updated_word_dict.npy'))
else:
    word_dict = list(np.load(f'{train_vect_path}word_dict.npy'))

In [5]:
len(word_dict)

35797

In [6]:
text = getFileText()

In [7]:
# print(len(text))

In [8]:
clear_old = False

if (clear_old): 
    vect_key = ''
    start_text_index = 0
    text_num=0
else: 
    old_np = np.load(f'{train_vect_path}train_vect.npy')
    vect_key='new_'
    start_text_index = old_np.shape[0]
    text_num = start_text_index

finished = False

In [9]:
if (not clear_old):
    new_np = np.load(f'{train_vect_path}{vect_key}train_vect.npy')
#     final_np = np.concatenate((old_np, new_np), axis=0)
    final_np = np.array(list(old_np) + list(new_np))
    np.save(f'{train_vect_path}train_vect.npy', final_np)
    print(f'{old_np.shape} + {new_np.shape} = {final_np.shape}')

(66974, 35799) + (9051, 35799) = (76025, 35799)


In [9]:
dict_indices = [i for i in range(len(word_dict))]

list_1 = []

total_texts = len(text)

if (start_text_index == 0): print('Starting at beginning of text corpus.')
else: print(f'Starting at article {start_text_index}/{total_texts}.')
    
start_lap = time.time()
for review_rating in text[start_text_index:]:
    
    if (len(review_rating) < 7): continue
    try:
        
        rr = review_rating.split('\n - \n')
        
        rating = rr[0]
        
        review = rr[1]

        vector = [0 for v in range(len(word_dict))]
                
        for i in review.split():
            try:
                word = trimWord(i).lower()
                
                if (word in stopWords): continue

                randomchar = False

                for c in randomchars: 
                    if (word.find(c) != -1): randomchar = True

                if (randomchar): continue
                    
                if (not hasVowel(word)): continue

                vector[word_dict.index(word)]+=1
                
            except:
                continue
            
            
        length = sum(vector)
        new_vector = [x / length for x in vector]
        
        
        if (int(rating)<=2): r = -1
        elif (int(rating)==3): r = 0
        elif (int(rating)>=4): r = 1
            
        list_1.append([int(r), length] + new_vector)

        text_num+=1
        if ((text_num-start_text_index)%1000==0):
                        
            end_lap = time.time()
            np_list = np.array(list_1)
            save_start = time.time()
            np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
            save_end = time.time()
            
            elapsed_time = end_lap - start_lap
            
            
            print(f'{text_num-start_text_index}/{total_texts-start_text_index} vectors saved', end = ' ')
            print(f'| {str(elapsed_time)[:6]} time units | Save Time {save_end-save_start} time units.')
            
            start_lap = time.time()
            
        if(text_num == total_texts-1): 
            np_list = np.array(list_1)
            
            np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
            print(f'{text_num+1-start_text_index}/{total_texts-start_text_index} vectors saved.')
            finished = True
        
    except: continue

if (not finished):
    np_list = np.array(list_1)
    np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
    print(f'{text_num}/{total_texts} vectors saved.')
    finished = True

Starting at article 66974/76036.
1000/9062 vectors saved | 4.9340 time units.
2000/9062 vectors saved | 6.0103 time units.
3000/9062 vectors saved | 5.6515 time units.
4000/9062 vectors saved | 4.4693 time units.
5000/9062 vectors saved | 5.1663 time units.
6000/9062 vectors saved | 7.5706 time units.
7000/9062 vectors saved | 11.240 time units.
8000/9062 vectors saved | 7.0490 time units.
9000/9062 vectors saved | 5.9354 time units.
76025/76036 vectors saved.


In [ ]:
if (not finished):
    np_list = np.array(list_1)
    np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
    print(f'{text_num}/{total_texts} vectors saved.')

###### remove words from the dictionary that do not appear much throughout the corpus

In [5]:
prune=True

In [6]:
if(prune):
    
    more_low_sums = False
    prune_df = pd.DataFrame(np.load(f'{train_vect_path}train_vect.npy'))

    sums=[1, 1]

    for i in range(2, prune_df.shape[1]):
        sums.append(prune_df[i].sum())
    
    threshold = 0.01
    low_sums = []
    
    for s in sums: 
        if (s<threshold): low_sums.append(s)
        
    if (more_low_sums):
        
        threshold1 = threshold*2
        threshold2 = threshold/2
        
        low_sums_1 = []
        low_sums_2 = []
        
        for s in sums:
            if (s<threshold1): low_sums_1.append(s)
            if (s<threshold2): low_sums_2.append(s)
            
        print(f'Threshold = {threshold1}: {len(low_sums_1)} values.')
        print(f'Threshold = {threshold}: {len(low_sums)} values.')
        print(f'Threshold2 = {threshold2}: {len(low_sums_2)} values.')
        
    else: print(f'Threshold = {threshold}: {len(low_sums)} values.')
    
    prune_df_sz = prune_df.shape[0]
    prune_df.loc[prune_df_sz] = sums
    
    columns_remove = (prune_df < threshold).loc[prune_df_sz, 2:]
    columns_remove = pd.concat([pd.Series([False, False]), columns_remove])
    prune_df = prune_df.drop(columns = prune_df.columns[columns_remove])
    
    
    update_word_dict = set()

    for word_index in (prune_df.columns.tolist()):
        word = word_dict[word_index-2]
        update_word_dict.add(word)

    update_word_dict_np = np.array(list(update_word_dict))
    np.save(f'{train_vect_path}updated_word_dict.npy', update_word_dict_np)
    print(f'Removed words, {update_word_dict_np.shape[0]}/{len(word_dict)} remain.')

Threshold = 0.01: 27011 values.
Removed words, 35797/62808 remain.
